<a href="https://colab.research.google.com/github/anmolsriv/COMP/blob/main/DBImpSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A5: Imperative SQL

The goal of this assignment is to develop skills writing functions and triggers. These will add functionality to our database. 

## What to turn in
You must turn in a Jupyter Notebook on Canvas. Basically, we want to be able to hit execute and run your code to create your imperative SQL and run it. **Make sure your cells run without error, even if you have no answer.** 
This  means that any comments or text answers in your file should be in SQL comments or in markdown cells. Answers in the comments section of assignment submissions on CANVAS are not acceptable and will not be read.

For your assignment to be graded, you must include:
* Your COMMENTED code (minimal or uncommented code will lose points)
* The results generated by your code
* DROP statements **at the end of your code** to clean up ALL VIEWs, FUNCTIONs, TRIGGERS, and TABLEs your code creates, excluding the tables we provide.



## Academic Honesty
The following level of collaboration is allowed on this assignment: 

You may discuss the assignment with your classmates at a high level. Any issues getting PostrgreSQL running is totally fine. What is not allowed is direct examination of anyone else's SQL code (on a computer, email, whiteboard, etc.) or allowing anyone else to see your SQL code.

You may use the search engine of your choice to look up the syntax for SQL commands or PostgreSQL imperative SQL syntax, but may not use it to find answers. Be sure to include the urls of any sites you reviewed as comments in your code.


## Grading
The number of points for each query is indicated in the question.   
If you don't get the right answer or your code is not correct, you won't get all of the points; partial credit may be given at the discretion of the grader.

This is intended to be an SQL assignment. You may use VIEWs as needed and you may use standard built-in PostgreSQL functions (e.g. `ROUND`, `IF`, or `CASE` statements). If you're not sure if something is allowed, ask!


In order for our autograder to work, we need to connect your solution to a local database. 

**DO NOT RUN CELLS THAT START WITH THE COMMENT `# LOCALDB` and do not remove those cells.**

For cells that contain your answer, replace the `SELECT 1` SQL code with your solution. 

**Do not change the first line of the cell or remove the following cell.  Do not add any text (comments, etc.) before the first line we provide.** If you do any of these things, the autograder will not work, we will have to manually modify your submission, and you will lose points.


Cells that start with:
`%%sql py_var_x <<`
Redirect the output of that cell to the variable named `py_var_x`. Note that if the next thing in this cell is a comment, you will get an error. Put some SQL code first, then add your comment. We then convert this variable to a dataframe, and record that as your answer for the question. You may add other cells to the notebook, but be sure that your answer is in the cell that copies the result to the appropriate Python variable.


We are using an autograder for the first pass grading. For the autograder to work, you need to complete your work in the appropriate cells and name and order the attributes in your results as directed. 

**It's critical that you use the attribute names and sort order as directed. If attribute name(s) are not specfied, use the name in the source table.**

* Use the function and trigger names specified
* Load data as provided
* We will be running the queries we gave you as well as additional queries that test how well your code works


Do not delete or modify the next 3 cells!

In [ ]:
# LOCALDB
import ipynbname
import psycopg2
from configparser import ConfigParser

def config(filename='.pg_service.conf', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

params = config()
# build the connection string
def make_conn_str(params):

    return f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}:{params['port']}/{params['dbname']}"
        
# connect to the database
conn_str = make_conn_str(params)
%load_ext sql
%sql $conn_str 
%config SqlMagic.displaylimit=100


# load the data
%system
myCmd = 'psql ' + params['user'] + ' -d ' +params['dbname'] + ' -p ' + params['port'] + ' -f DBImpSQLdata.sql'
!{myCmd}

In [ ]:
# LOCALDB
nb_fname = ipynbname.name()
myInsert = "INSERT INTO studentNotebook(name) VALUES ( '{}' );".format(nb_fname)
print(myInsert)
myCmd = 'psql ' + params['user'] + ' -d ' +params['dbname'] + ' -p ' + params['port']  + ' -c "' + myInsert + '"'
!{myCmd}


## Initialization

The next cell needs to be run each time you start up Google Colab to start up PostgreSQL.

In [ ]:
# install
!pip install --upgrade pip
!pip install SQLAlchemy==1.3.23
!pip install psycopg2-binary
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.autolimit=100
# Limit queries to 100 results. Increase this value if needed, but recognize that your notebook will increase in size as well. %config SqlMagic.displaylimit=100
%sql postgresql+psycopg2://@/postgres
!sudo -u postgres createdb ricedb

The following tables are used in this assignment:

* Assembly(<ins>assemblyId</ins>, productCode, ingName, qty, unit, hasChoice, detail)
    - Assigns ingredients to products
* Ingredient(<ins>ingName</ins>)
    - Contains ingredients that combine into products
* IngChoice(<ins>ingName</ins>, <ins>choiceName</ins>)
    - Contains ingredients that have user selected choices
* Menu(<ins>menuName</ins>)
    - Contains name of the menus
* MenuProduct(<ins>menuName</ins>, <ins>productCode</ins>, price)
    - Associates products to menus with a price for each product on that menu. A single product may be priced differently on different menus.
* Msg(<ins>id</ins>, occurred, msg) 
    - Table in which values can be written for debugging purposes* Product(<ins>productCode</ins>, productName)
    - Contains the names and codes for products
* ProductSold(<ins>productSoldId</ins>, productCode, ticketId, price)
    - Information about the product that was actually sold
* SaleRecord(<ins>saleRecordId</ins>, productSoldId, ingName, qty, unit, detail)
    - Information about each ingredient used in ProductSold
* Ticket(<ins>ticketId</ins>, eventId, ticketTime, numProducts, totalPrice)
    - Contains products sold together by one purchaser
* TruckEvent(<ins>eventId</ins>, eventName, eventStart, eventPlannedEnd, eventActualEnd, eventStatus, menuName totalDollarSales, totalNumProducts, totalNumTickets) 
    - Contains events that the ice cream truck goes to


## Load the data

In [ ]:
# create the tables and load the starting data
!psql postgres -f DBImpSQLdata.sql

### Declarative SQL

1.  (3 points) Write a query that returns the ingredients used in the product named "brain freeze". For each ingredient, return the ingredient name, detail, quantity, unit, whether or not the ingredient requires a choice and all possible choices. Sort by ingredient name then choice name. 

Each ingredient with a choice will be repeated with each choice option. For example, if a product includes ice cream, and the choices are Chocolate, Vanilla, and Strawberry, your query should return 3 records for ice cream in that product.

Show your query results. 

In [ ]:
%%sql py_var_IngUsed <<
SELECT 1; -- your code here

In [ ]:
# do not modify or delete this cell
df_q01 = py_var_IngUsed.DataFrame() # Save the result of this task for grading purposes
df_q01

2. (3 points) We want to know how much of our inventory we used during an event.

Create a VIEW named `eventIngredient` that returns all of the ingredients used by event, grouped by event, ingName, unit, and detail that shows the number of each ingredient consumed and the total quantity.  This should be a list with a line for every ingredient option that was used during each event. 

Attributes, should include, in this order:

* eventId
* ingName
* unit
* detail
* total number of that ingredient (named totalNumber)
* total quantity of that ingredient (named totalQty)

In [ ]:
%%sql
CREATE OR REPLACE VIEW eventIngredient AS
SELECT 1 -- your code here;

## Functions

3. (6 points) Create a function, ``createTicket`` that:
		* Takes as parameters a TruckEvent eventId
		* Creates an empty ticket with the current timestamp
		* Returns the id of the new ticket
		* If there is no event with the specified eventId, returns -1

Notes: Recall that you can use  the keyword `RETURNING` as part of an `INSERT` statement to get back the newly assigned ticketId. See https://www.postgresql.org/docs/10/sql-insert.html for more details.


In [ ]:
%%sql 
CREATE OR REPLACE FUNCTION createTicket ( thisEventId INTEGER) 
  RETURNS INTEGER AS  
$$
BEGIN
-- your code here, be sure to update the return value
RETURN 0;
END;
$$
LANGUAGE plpgsql;

  

4. (9 points) Write a function, `addProductToTicket`, that takes a ticketId and a productCode and adds the specified product to the specified ticket and returns the id of the new ProductSold record. Be sure to populate all of the appropriate fields for ProductSold (e.g. price).   This function should add all the ingredients **without** choices to the saleRecord table. 

If the product specified is not on the menu for this event, print the message ''Product <XX> is not available'', where XX is the product specified, and return -1. Also return -1 if the ticketId is not valid.

In [ ]:
%%sql 
CREATE OR REPLACE FUNCTION addProductToTicket ( thisTicketId INTEGER, thisProductCode CHAR(3)) 
  RETURNS INTEGER AS  
$$
BEGIN
-- your code here, be sure to update the return value
RETURN 0;
END;
$$
LANGUAGE plpgsql;

  

5. (10 points) Write a function, `addIngChoiceToProductSold`, that takes a productSoldId, an ingredient name, a quantity, a unit, and a choice name and adds the chosen ingredient to the specified productSold. Be sure to populate all of the fields for SaleRecord. If an invalid productSoldId is passed in or the specified ingredient is not part of the product, return -1. Otherwise, return the id of the new SaleRecord.

In [ ]:
%%sql 
CREATE OR REPLACE FUNCTION addIngChoiceToProductSold ( thisProductSoldId INTEGER, thisIngName VARCHAR(50), 
                                                      thisQty NUMERIC(10,2), thisUnit VARCHAR(20), thisChoice VARCHAR(50)) 
  RETURNS INTEGER AS
$$
BEGIN
-- your code here, be sure to update the return value
RETURN 0;
END;
$$
LANGUAGE plpgsql;

  

6. (5 points)  Write a function named `updateEventSales` that takes a truckEvent eventId and updates totalDollarSales, totalNumProducts, and totalNumTickets based on the products sold during the event.   Returns the eventId if successful. Returns -1 if the eventId does not exist.

In [ ]:
%%sql 
CREATE OR REPLACE FUNCTION updateEventSales ( thisEventId INTEGER) 
  RETURNS INTEGER AS  
$$
BEGIN
-- your code here, be sure to update the return value
RETURN 0;
END;
$$
LANGUAGE plpgsql;

  

7. (25 points) Write a function ``findLongestChain`` that takes an eventId and a product code. ``findLongestChain`` should look at tickets from the specified event in order by their  ticketTime and return the maximum number of tickets in a row that has at least one of the specified products.  You should not make any assumptions about the assignment of productSoldId values. Return -1 if the eventId is invalid. 

For example, say tickets 101, 102, and 103 occured during event 1.

Ticket 101 contains: c1, c2, and c3.

Ticket 102 contains: c2, ms, and dk.

Ticket 103 contains: dk and c1.

If we are looking for the longest chain for product `dk`, the answer would be 2.
The longest chain for c1 is 1.

In [ ]:
%%sql 
CREATE OR REPLACE FUNCTION findLongestChain (  thisEventId INTEGER, thisProductCode CHAR(3)) 
  RETURNS INTEGER AS
$$
BEGIN
-- your code here, be sure to update the return value
RETURN 0;
END;
$$
LANGUAGE plpgsql;

  

## Triggers

8. (10 points)  Write a trigger named `updateTicket` on the productSold table that  updates the numProducts and totalPrice fields in Ticket when a product is added to the productSold table

In [ ]:
%%sql 
SELECT 1; -- your trigger here

## Putting it all together with Transactions

9. (1 point) Create an event 'DB study group' with start date time December 1, 2021 at 8 PM and planned end time December 2, 2021 at 12 AM. The status should be 'Scheduled' and the menu should be 'Full menu'.

Assign this event eventId 5. Note that you can assign values to attributes of type `SERIAL` as long as that value does not already exist in the table.


In [ ]:
%%sql 
SELECT 1; -- your code here 

10. Run your function ```updateEventSales``` for this event to initialize the derived attributes.

In [ ]:
%%sql
SELECT updateEventSales(5)

11. (14 points) Create a transaction that includes the following actions:

1 Create a new ticket for this event. Keep track of the unique id for this ticket. There are numerous ways to do this programmatically. A simple way is to create a variable and put the result of your function and / or query into it.
    
2 Insert the message "ticketId X" where X is the id of your new ticket into Msg

3 Add a Brain Freeze product to this ticket. The choices for this product should include `vanilla` ice cream and `cherry` flavoring. You may programmatically or manually (visually) look up the product code and ingredient information (qty, unit, etc.) for a Brain Freeze. Keep track of the unique id of this product sold.
    
4 Insert the message "brain freeze X" where X is the id of your the productSold for the brain freeze into Msg.
    
5 Add a regular slush product with "grape" flavor.  You may programmatically or manually look up the product code  and ingredient information (qty, unit, etc.) for a regular slush.
    
6 Insert the message "regular slush X" where X is the id of your the productSold for the regular slush into Msg.
    
7 Commit the transaction.    

In [ ]:
%%sql 
SELECT 1; -- your code here 

12. (2 points for correct messages) Show us only the messages in your msg table, order by the time in which they were inserted.

In [ ]:
%%sql py_var_msg <<
SELECT msg
FROM Msg
ORDER BY occurred;

In [ ]:
# do not modify or delete this cell
df_q02 = py_var_msg.DataFrame() # Save the result of this task for grading purposes
df_q02

## Run grading script

Leave the next cell in your notebook. We will use it to run our grading script.

In [ ]:
%system
myCmd = 'psql ' + params['user'] + ' -d ' +params['dbname'] + ' -p ' + params['port'] + ' -o impSQLoutput.txt -f impSQLgrader.sql'
!{myCmd}

12. (9 points) Describe and provide SQL statements for 3 different test cases for `findLongestChain`, different from the examples provided above. Explain what each test case tests. 

Test case 1 description:

In [ ]:
%%sql 
SELECT 1;
-- your code here 

Test case 2 description:

In [ ]:
%%sql 
SELECT 1;
-- your code here 

Test case 3 description:

In [ ]:
%%sql 
SELECT 1;
-- your code here 

13. (5 points) Add code to drop functions, custom views, new tables, and triggers that you created

Be sure to put the drop statements in the correct order (or use CASCADE)!


In [ ]:
%%sql 
SELECT 1;
-- your code here 